# Assignment 3
(Assignment groups 2 and 10)

- Jasper Bosschart s2562685
- Marina Stefanova s2610604
- Nagham Mekky s3096564
- Sven Sonneveld s2594854
- Hynek Noll s3102181

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import csv
import pandas as pd

# Motivation and Goals

# Methodology and Results 
Our goal is to analyze leaked chat logs of a ransomware group Conti.

1) How many total records are in the data?  

In [6]:
file = open('jabber_chat_2021_2022_translated.csv',encoding='utf8')
reader = csv.reader(file)
record_count= len(list(reader))-1
print("Number of records: ", record_count)

Number of records:  60773


2) How many of these records are encrypted and cannot be read?

3) How many messages are sent originally in Russian, and how many are sent in English?

In [4]:
russian_count = 0
english_count = 0
# Read the data from the csv file
with open('jabber_chat_2021_2022_translated.csv','r',encoding='utf8') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for row in lines:
        if (row[6]=='ru'):
            russian_count += 1
        elif (row[6]=='en'):
            english_count += 1
print("Number of Russian messages: ", russian_count)
print("Number of English messages: ", english_count)

Number of Russian messages:  32706
Number of English messages:  4314


In [13]:
dict = {}
with open('jabber_chat_2021_2022_translated.csv','r',encoding='utf8') as csvfile:
    lines = csv.reader(csvfile, delimiter=',')
    for elem in lines:
        if elem[2] not in dict:
            dict[elem[2]] = 0
        dict[elem[2]] = dict[elem[2]] + 1

final_list = [{'Email address' : elem, 'count': dict[elem]} for elem in dict]
print (final_list)

[{'Email address': 'from', 'count': 1}, {'Email address': 'driver@q3mcco35auwcstmt.onion', 'count': 3968}, {'Email address': 'hof@q3mcco35auwcstmt.onion', 'count': 2389}, {'Email address': 'bentley@q3mcco35auwcstmt.onion', 'count': 1810}, {'Email address': 'tom@q3mcco35auwcstmt.onion', 'count': 393}, {'Email address': 'mentos@q3mcco35auwcstmt.onion', 'count': 67}, {'Email address': 'strix@q3mcco35auwcstmt.onion', 'count': 265}, {'Email address': 'merch@q3mcco35auwcstmt.onion', 'count': 39}, {'Email address': 'defender@q3mcco35auwcstmt.onion', 'count': 8246}, {'Email address': 'professor@q3mcco35auwcstmt.onion', 'count': 189}, {'Email address': 'carter@q3mcco35auwcstmt.onion', 'count': 123}, {'Email address': 'love@q3mcco35auwcstmt.onion', 'count': 552}, {'Email address': 'sunday@q3mcco35auwcstmt.onion', 'count': 472}, {'Email address': 'grom@q3mcco35auwcstmt.onion', 'count': 1200}, {'Email address': 'derek@q3mcco35auwcstmt.onion', 'count': 255}, {'Email address': 'miguel@q3mcco35auwcst

# Conclusion and Reflections